In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-1.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-1        4    2    4    0    2    5     1   0   2  ...  1.000000   
1      prop-1        4   12    4    1   19   44    66   2  18  ...  0.000000   
2      prop-1        4   10    4    0   13   48    45   4  13  ...  1.000000   
3      prop-1        4    2    3    0    5    7     1   0   5  ...  1.000000   
4      prop-1        4    8    1    0    4   38     0   3   1  ...  0.500000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
18466  prop-1      185   17    1    0   24   67    96   9  15  ...  0.150000   
18467  prop-1      185    9    1    0   25   49    14   7  18  ...  0.333333   
18468  prop-1      185    6    3    0   10   29     0   0  10  ...  0.800000   
18469  prop-1      185    5    5    0   28    9    10  26   2  ...  1.000000   
18470  prop-1      185   11    1    0   54   34     9  51   3  ...  0.666667   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0        0  0.941176  0.750000   1    1  10.500000       3  1.5000    0  
1        0  0.905172  0.350000   2    3  20.583333       8  2.0833    0  
2        0  0.918919  0.500000   2    3  19.100000       4  1.2000    0  
3        0  0.933333  0.750000   1    1  11.500000       4  2.0000    1  
4        0  0.000000  0.285714   0    0  30.625000       2  0.8750    0  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
18466    0  0.000000  0.250000   0    0  41.529412       6  2.6471    1  
18467    0  0.000000  0.319444   0    0  29.888889       3  1.3333    0  
18468    0  0.818182  0.400000   0    0  45.500000       6  2.1667    0  
18469    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
18470    1  0.000000  0.400000   0    0  25.545455       6  1.9091    0  

[18471 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 157]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...  dam  moa  \
13150  prop-1      157    9    5    0   27   45    32   5  23  ...  0.0    0   
13151  prop-1      157    3    4    0    1    4     3   0   1  ...  0.0    0   
13152  prop-1      157   12    2    0   76   24    54  71   5  ...  1.0    0   
13153  prop-1      157    2    1    0    3    2     1   3   0  ...  0.0    0   
13154  prop-1      157    2    1    0   12    2     1  12   0  ...  0.0    0   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...  ...  ...   
15641  prop-1      157    3    3    0    8   11     1   0   8  ...  1.0    0   
15642  prop-1      157    4    2    0    5    6     6   0   5  ...  0.0    0   
15643  prop-1      157    8    5    0   15   12    22  13   2  ...  1.0    0   
15644  prop-1      157    1    1    0    4    1     0   2   2  ...  0.0    0   
15645  prop-1      157    8    1    0   25   44    16   9  16  ...  1.0    0   

            mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
13150  0.931034  0.333333   2    7  18.777778       5  1.3333    4  
13151  0.800000  1.000000   1    1   2.333333       1  0.6667    0  
13152  0.500000  0.226190   1    1   7.833333       1  0.9167    0  
13153  0.000000  1.000000   0    0   0.000000       1  1.0000    0  
13154  0.000000  1.000000   0    0   0.000000       1  1.0000    0  
...         ...       ...  ..  ...        ...     ...     ...  ...  
15641  0.947368  0.666667   0    0   9.000000       2  0.6667    0  
15642  0.571429  0.562500   1    1   2.750000       1  0.7500    0  
15643  0.928571  0.575000   1    1   5.250000       1  0.2500    0  
15644  0.000000  1.000000   0    0   0.000000       1  1.0000    0  
15645  0.000000  0.457143   0    0  39.250000       2  1.1250    0  

[2496 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc  dam  moa  \
14488    2    3    0   26   30     1   0  26    2  2.000000  256  0.0    0   
14701   11    1    1   16   56    33   1  15    6  1.053488  553  1.0    2   
15115    8    1    0    3   27     0   2   1    7  0.657143  136  1.0    1   
14052   15    1    0    1   19    87   0   1   15  0.892857   72  1.0    0   
14741    3    1    0    2    5     0   2   0    3  0.500000   24  1.0    0   
...    ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...  ...  ...   
15359    3    3    0    4    6     3   0   4    3  2.000000   14  0.0    0   
15401    2    2    0    7    7     1   0   7    2  2.000000   20  0.0    0   
15264    3    3    0    4    6     3   0   4    3  2.000000   14  0.0    0   
13673    1    1    0    0    2     0   0   0    0  2.000000   12  0.0    0   
13515    7    5    0    8   16    21   0   8    1  2.000000   51  0.0    0   

            mfa       cam  ic  cbm         amc  max_cc   avg_cc  
14488  0.857143  0.666667   2    2  127.000000      24  12.0000  
14701  0.000000  0.480000   0    0   45.363636       5   1.8182  
15115  0.000000  0.428571   0    0   15.375000       3   1.1250  
14052  0.000000  0.444444   0    0    3.533333       1   0.9333  
14741  0.000000  1.000000   0    0    6.333333       1   0.6667  
...         ...       ...  ..  ...         ...     ...      ...  
15359  0.777778  0.583333   1    1    3.666667       1   0.6667  
15401  0.800000  0.666667   1    1    9.000000       1   0.5000  
15264  0.777778  0.583333   1    1    3.666667       1   0.6667  
13673  0.000000  0.000000   0    0    4.000000       0   0.0000  
13515  0.944954  0.500000   2    5    6.285714       1   0.8571  

[1996 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc  dam  moa  \
13184    3    4    0    6    7     3   0   6    2  2.000000   17  0.0    0   
13493    7    1    0    1    8     3   0   1    7  0.666667   43  1.0    0   
15602    4    3    0   13   45     6   1  13    2  2.000000  252  0.0    0   
14584    5    4    2    3   10    10   2   1    5  2.000000   29  0.0    0   
14424    3    1    0   10   46     1   0  10    1  0.500000  216  1.0    0   
...    ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...  ...  ...   
13456    2    3    0    5    6     1   0   5    1  2.000000   27  1.0    0   
14325    2    3    0    3    6     1   0   3    1  2.000000   26  1.0    0   
14767    9    4    0    7   13    34   0   7    9  0.968750   51  1.0    0   
15250    2    2    0   11    9     1   0  11    2  2.000000   30  0.0    0   
15515    2    3    0   11   14     1   0  11    2  2.000000  116  0.0    0   

            mfa       cam  ic  cbm        amc  max_cc  avg_cc  
13184  0.970149  1.000000   1    1   4.666667       1  0.6667  
13493  0.000000  0.714286   0    0   4.714286       1  0.8571  
15602  0.842105  0.500000   2    6  62.000000       9  3.5000  
14584  1.000000  0.700000   0    0   4.800000       0  0.0000  
14424  0.000000  0.500000   0    0  69.000000      12  4.0000  
...         ...       ...  ..  ...        ...     ...     ...  
13456  0.928571  0.750000   1    1  12.000000       3  1.5000  
14325  0.928571  0.750000   1    1  11.500000       4  2.0000  
14767  0.859649  1.000000   0    0   4.222222       2  1.0000  
15250  0.800000  0.666667   1    1  14.000000       1  0.5000  
15515  0.947368  0.600000   0    0  57.000000       1  0.5000  

[500 rows x 20 columns]

In [9]:
y_train = train['bug']
y_test = test['bug']

In [10]:
y_train

14488    0
14701    0
15115    0
14052    0
14741    0
        ..
15359    0
15401    0
15264    0
13673    0
13515    0
Name: bug, Length: 1996, dtype: int64

In [11]:
y_test

13184    0
13493    0
15602    0
14584    0
14424    0
        ..
13456    0
14325    0
14767    0
15250    0
15515    0
Name: bug, Length: 500, dtype: int64

In [12]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [13]:
x_train_np

array([[  2.        ,   3.        ,   0.        , ..., 127.        ,
         24.        ,  12.        ],
       [ 11.        ,   1.        ,   1.        , ...,  45.36363636,
          5.        ,   1.8182    ],
       [  8.        ,   1.        ,   0.        , ...,  15.375     ,
          3.        ,   1.125     ],
       ...,
       [  3.        ,   3.        ,   0.        , ...,   3.66666667,
          1.        ,   0.6667    ],
       [  1.        ,   1.        ,   0.        , ...,   4.        ,
          0.        ,   0.        ],
       [  7.        ,   5.        ,   0.        , ...,   6.28571429,
          1.        ,   0.8571    ]])

In [14]:
x_test_np

array([[ 3.        ,  4.        ,  0.        , ...,  4.66666667,
         1.        ,  0.6667    ],
       [ 7.        ,  1.        ,  0.        , ...,  4.71428571,
         1.        ,  0.8571    ],
       [ 4.        ,  3.        ,  0.        , ..., 62.        ,
         9.        ,  3.5       ],
       ...,
       [ 9.        ,  4.        ,  0.        , ...,  4.22222222,
         2.        ,  1.        ],
       [ 2.        ,  2.        ,  0.        , ..., 14.        ,
         1.        ,  0.5       ],
       [ 2.        ,  3.        ,  0.        , ..., 57.        ,
         1.        ,  0.5       ]])

In [15]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [16]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [17]:
train_x

array([[[  2.        ,   3.        ,   0.        , ..., 127.        ,
          24.        ,  12.        ]],

       [[ 11.        ,   1.        ,   1.        , ...,  45.36363636,
           5.        ,   1.8182    ]],

       [[  8.        ,   1.        ,   0.        , ...,  15.375     ,
           3.        ,   1.125     ]],

       ...,

       [[  3.        ,   3.        ,   0.        , ...,   3.66666667,
           1.        ,   0.6667    ]],

       [[  1.        ,   1.        ,   0.        , ...,   4.        ,
           0.        ,   0.        ]],

       [[  7.        ,   5.        ,   0.        , ...,   6.28571429,
           1.        ,   0.8571    ]]])

In [18]:
test_y

array([0, 0, 0, 0, 0, 0, 1, 0, 3, 3, 0, 1, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 3, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 8, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 8, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 2,

In [19]:
def define_model():
    # Designing and initializing the model.
    model = Sequential()
    model.add(LSTM(200, return_sequences=True))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(50, dropout = 0.2, return_sequences=True))
    model.add(LSTM(20, return_sequences=False))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )
    return model

In [20]:
y = list(test_y)

In [21]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(10):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/propv157/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(test_x, test_y)
    total_scores.append(score) 
    predictions = model.predict(test_x)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
20/20 [==============================] - 7s 31ms/step - loss: 0.9166 - mse: 0.9166 - mae: 0.2946 - root_mean_squared_error: 0.9574 - mean_squared_logarithmic_error: 0.1604
Epoch 2/100
20/20 [==============================] - 0s 22ms/step - loss: 0.8609 - mse: 0.8609 - mae: 0.3876 - root_mean_squared_error: 0.9279 - mean_squared_logarithmic_error: 0.1458
Epoch 3/100
20/20 [==============================] - 0s 21ms/step - loss: 0.8178 - mse: 0.8178 - mae: 0.4580 - root_mean_squared_error: 0.9043 - mean_squared_logarithmic_error: 0.1474
Epoch 4/100
20/20 [==============================] - 0s 21ms/step - loss: 0.7866 - mse: 0.7866 - mae: 0.4253 - root_mean_squared_error: 0.8869 - mean_squared_logarithmic_error: 0.1401
Epoch 5/100
20/20 [==============================] - 0s 21ms/step - loss: 0.7633 - mse: 0.7633 - mae: 0.4182 - root_mean_squared_error: 0.8737 - mean_squared_logarithmic_error: 0.1371
Epoch 6/100
20/20 [==============================] - 0s 21ms/step - lo

20/20 [==============================] - 0s 21ms/step - loss: 0.4521 - mse: 0.4521 - mae: 0.3156 - root_mean_squared_error: 0.6724 - mean_squared_logarithmic_error: 0.0948
Epoch 89/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4210 - mse: 0.4210 - mae: 0.3182 - root_mean_squared_error: 0.6488 - mean_squared_logarithmic_error: 0.0921
Epoch 90/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4420 - mse: 0.4420 - mae: 0.3231 - root_mean_squared_error: 0.6648 - mean_squared_logarithmic_error: 0.0950
Epoch 91/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4268 - mse: 0.4268 - mae: 0.3229 - root_mean_squared_error: 0.6533 - mean_squared_logarithmic_error: 0.0917
Epoch 92/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4101 - mse: 0.4101 - mae: 0.3079 - root_mean_squared_error: 0.6404 - mean_squared_logarithmic_error: 0.0903
Epoch 93/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4218 - mse:

20/20 [==============================] - 0s 22ms/step - loss: 0.4684 - mse: 0.4684 - mae: 0.3170 - root_mean_squared_error: 0.6844 - mean_squared_logarithmic_error: 0.1006
Epoch 74/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4426 - mse: 0.4426 - mae: 0.3033 - root_mean_squared_error: 0.6653 - mean_squared_logarithmic_error: 0.0886
Epoch 75/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4524 - mse: 0.4524 - mae: 0.3283 - root_mean_squared_error: 0.6726 - mean_squared_logarithmic_error: 0.0974
Epoch 76/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4377 - mse: 0.4377 - mae: 0.3133 - root_mean_squared_error: 0.6616 - mean_squared_logarithmic_error: 0.0920
Epoch 77/100
20/20 [==============================] - 0s 22ms/step - loss: 0.4450 - mse: 0.4450 - mae: 0.3183 - root_mean_squared_error: 0.6671 - mean_squared_logarithmic_error: 0.0938
Epoch 78/100
20/20 [==============================] - 0s 22ms/step - loss: 0.4395 - mse:

20/20 [==============================] - 0s 22ms/step - loss: 0.4846 - mse: 0.4846 - mae: 0.3152 - root_mean_squared_error: 0.6961 - mean_squared_logarithmic_error: 0.0965
Epoch 59/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4818 - mse: 0.4818 - mae: 0.3317 - root_mean_squared_error: 0.6941 - mean_squared_logarithmic_error: 0.1006
Epoch 60/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4794 - mse: 0.4794 - mae: 0.3325 - root_mean_squared_error: 0.6924 - mean_squared_logarithmic_error: 0.1013
Epoch 61/100
20/20 [==============================] - 0s 22ms/step - loss: 0.4710 - mse: 0.4710 - mae: 0.3200 - root_mean_squared_error: 0.6863 - mean_squared_logarithmic_error: 0.0971
Epoch 62/100
20/20 [==============================] - 0s 22ms/step - loss: 0.4807 - mse: 0.4807 - mae: 0.3297 - root_mean_squared_error: 0.6933 - mean_squared_logarithmic_error: 0.0997
Epoch 63/100
20/20 [==============================] - 0s 22ms/step - loss: 0.4596 - mse:

Epoch 1/100
20/20 [==============================] - 6s 40ms/step - loss: 0.9006 - mse: 0.9006 - mae: 0.3163 - root_mean_squared_error: 0.9490 - mean_squared_logarithmic_error: 0.1540
Epoch 2/100
20/20 [==============================] - 0s 21ms/step - loss: 0.8400 - mse: 0.8400 - mae: 0.4381 - root_mean_squared_error: 0.9165 - mean_squared_logarithmic_error: 0.1476
Epoch 3/100
20/20 [==============================] - 0s 21ms/step - loss: 0.8116 - mse: 0.8116 - mae: 0.4480 - root_mean_squared_error: 0.9009 - mean_squared_logarithmic_error: 0.1446
Epoch 4/100
20/20 [==============================] - 0s 22ms/step - loss: 0.7906 - mse: 0.7906 - mae: 0.4296 - root_mean_squared_error: 0.8892 - mean_squared_logarithmic_error: 0.1417
Epoch 5/100
20/20 [==============================] - 0s 21ms/step - loss: 0.7738 - mse: 0.7738 - mae: 0.4099 - root_mean_squared_error: 0.8797 - mean_squared_logarithmic_error: 0.1372
Epoch 6/100
20/20 [==============================] - 0s 22ms/step - loss: 0.7509

...

20/20 [==============================] - 0s 21ms/step - loss: 0.4250 - mse: 0.4250 - mae: 0.3150 - root_mean_squared_error: 0.6519 - mean_squared_logarithmic_error: 0.0939
Epoch 89/100
20/20 [==============================] - 0s 22ms/step - loss: 0.4750 - mse: 0.4750 - mae: 0.3083 - root_mean_squared_error: 0.6892 - mean_squared_logarithmic_error: 0.0961
Epoch 90/100
20/20 [==============================] - 0s 22ms/step - loss: 0.4230 - mse: 0.4230 - mae: 0.3052 - root_mean_squared_error: 0.6504 - mean_squared_logarithmic_error: 0.0884
Epoch 91/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4566 - mse: 0.4566 - mae: 0.3110 - root_mean_squared_error: 0.6757 - mean_squared_logarithmic_error: 0.0950
Epoch 92/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4274 - mse: 0.4274 - mae: 0.3101 - root_mean_squared_error: 0.6537 - mean_squared_logarithmic_error: 0.0928
Epoch 93/100
20/20 [==============================] - 0s 22ms/step - loss: 0.4552 - mse:

20/20 [==============================] - 0s 21ms/step - loss: 0.4745 - mse: 0.4745 - mae: 0.3134 - root_mean_squared_error: 0.6889 - mean_squared_logarithmic_error: 0.0936
Epoch 74/100
20/20 [==============================] - 0s 22ms/step - loss: 0.4793 - mse: 0.4793 - mae: 0.3280 - root_mean_squared_error: 0.6923 - mean_squared_logarithmic_error: 0.0966
Epoch 75/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4812 - mse: 0.4812 - mae: 0.3201 - root_mean_squared_error: 0.6937 - mean_squared_logarithmic_error: 0.0989
Epoch 76/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4744 - mse: 0.4744 - mae: 0.3036 - root_mean_squared_error: 0.6888 - mean_squared_logarithmic_error: 0.0933
Epoch 77/100
20/20 [==============================] - 0s 23ms/step - loss: 0.5104 - mse: 0.5104 - mae: 0.3312 - root_mean_squared_error: 0.7144 - mean_squared_logarithmic_error: 0.0999
Epoch 78/100
20/20 [==============================] - 0s 23ms/step - loss: 0.4873 - mse:

20/20 [==============================] - 0s 22ms/step - loss: 0.5067 - mse: 0.5067 - mae: 0.3510 - root_mean_squared_error: 0.7119 - mean_squared_logarithmic_error: 0.1059
Epoch 59/100
20/20 [==============================] - 0s 22ms/step - loss: 0.5118 - mse: 0.5118 - mae: 0.3391 - root_mean_squared_error: 0.7154 - mean_squared_logarithmic_error: 0.1021
Epoch 60/100
20/20 [==============================] - 0s 22ms/step - loss: 0.4930 - mse: 0.4930 - mae: 0.3332 - root_mean_squared_error: 0.7021 - mean_squared_logarithmic_error: 0.0996
Epoch 61/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4765 - mse: 0.4765 - mae: 0.3310 - root_mean_squared_error: 0.6903 - mean_squared_logarithmic_error: 0.1010
Epoch 62/100
20/20 [==============================] - 0s 22ms/step - loss: 0.4647 - mse: 0.4647 - mae: 0.3236 - root_mean_squared_error: 0.6817 - mean_squared_logarithmic_error: 0.0956
Epoch 63/100
20/20 [==============================] - 0s 22ms/step - loss: 0.4597 - mse:

Epoch 1/100
20/20 [==============================] - 7s 41ms/step - loss: 0.9019 - mse: 0.9019 - mae: 0.3137 - root_mean_squared_error: 0.9497 - mean_squared_logarithmic_error: 0.1541
Epoch 2/100
20/20 [==============================] - 0s 21ms/step - loss: 0.8381 - mse: 0.8381 - mae: 0.4392 - root_mean_squared_error: 0.9155 - mean_squared_logarithmic_error: 0.1480
Epoch 3/100
20/20 [==============================] - 0s 21ms/step - loss: 0.8058 - mse: 0.8058 - mae: 0.4517 - root_mean_squared_error: 0.8976 - mean_squared_logarithmic_error: 0.1450
Epoch 4/100
20/20 [==============================] - 0s 22ms/step - loss: 0.7763 - mse: 0.7763 - mae: 0.4179 - root_mean_squared_error: 0.8811 - mean_squared_logarithmic_error: 0.1388
Epoch 5/100
20/20 [==============================] - 0s 22ms/step - loss: 0.7502 - mse: 0.7502 - mae: 0.4150 - root_mean_squared_error: 0.8661 - mean_squared_logarithmic_error: 0.1361
Epoch 6/100
20/20 [==============================] - 0s 22ms/step - loss: 0.7285

20/20 [==============================] - 0s 20ms/step - loss: 0.5001 - mse: 0.5001 - mae: 0.3277 - root_mean_squared_error: 0.7072 - mean_squared_logarithmic_error: 0.0985
Epoch 90/100
20/20 [==============================] - 0s 22ms/step - loss: 0.4407 - mse: 0.4407 - mae: 0.3171 - root_mean_squared_error: 0.6639 - mean_squared_logarithmic_error: 0.0920
Epoch 91/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4428 - mse: 0.4428 - mae: 0.3178 - root_mean_squared_error: 0.6655 - mean_squared_logarithmic_error: 0.0926
Epoch 92/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4258 - mse: 0.4258 - mae: 0.2965 - root_mean_squared_error: 0.6526 - mean_squared_logarithmic_error: 0.0878
Epoch 93/100
20/20 [==============================] - 0s 22ms/step - loss: 0.4498 - mse: 0.4498 - mae: 0.3115 - root_mean_squared_error: 0.6706 - mean_squared_logarithmic_error: 0.0938
Epoch 94/100
20/20 [==============================] - 0s 21ms/step - loss: 0.4522 - mse:

In [22]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/propv157/'

In [23]:
total_training_time

[55.72205901145935,
 59.58463644981384,
 56.154752254486084,
 56.153205156326294,
 57.41757011413574,
 56.998733043670654,
 56.21480417251587,
 58.55304169654846,
 58.299643754959106,
 56.81039094924927]

In [24]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 19.063627886772156


In [25]:
total_training_loss_history

[{'loss': [0.9165698289871216,
   0.860944926738739,
   0.8177658915519714,
   0.7865939736366272,
   0.7633220553398132,
   0.7346040606498718,
   0.7068659067153931,
   0.7020308375358582,
   0.6873377561569214,
   0.6880228519439697,
   0.6662464141845703,
   0.6670944094657898,
   0.6604594588279724,
   0.6431227326393127,
   0.6395556330680847,
   0.6409285068511963,
   0.6556226015090942,
   0.6616753935813904,
   0.6312131881713867,
   0.603468656539917,
   0.6266433596611023,
   0.5997073650360107,
   0.6281626224517822,
   0.5940402746200562,
   0.5882191061973572,
   0.5705480575561523,
   0.578079104423523,
   0.5701292753219604,
   0.5619948506355286,
   0.5635517835617065,
   0.5507245063781738,
   0.5705042481422424,
   0.5660882592201233,
   0.5562350749969482,
   0.5501314997673035,
   0.557786762714386,
   0.535342276096344,
   0.539345920085907,
   0.5711452960968018,
   0.5406677722930908,
   0.5223909020423889,
   0.5395960807800293,
   0.5559984445571899,
   0.5206

In [26]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(10):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 0.14180134534835814
avg mae of training: 0.10145614643891653


In [27]:
total_scores

[[0.7474979162216187,
  0.7474979162216187,
  0.3753111958503723,
  0.8645796179771423,
  0.13422891497612],
 [0.8510481715202332,
  0.8510481715202332,
  0.3893231153488159,
  0.9225227236747742,
  0.13983950018882751],
 [0.8214856386184692,
  0.8214856386184692,
  0.4073634445667267,
  0.9063584208488464,
  0.14273813366889954],
 [0.8583977818489075,
  0.8583977818489075,
  0.3952447474002838,
  0.9264975786209106,
  0.14156056940555573],
 [0.8215423226356506,
  0.8215423226356506,
  0.43463245034217834,
  0.9063897132873535,
  0.15519174933433533],
 [0.9639779329299927,
  0.9639779329299927,
  0.43296703696250916,
  0.9818238019943237,
  0.15869006514549255],
 [0.7724539637565613,
  0.7724539637565613,
  0.40090125799179077,
  0.8788936138153076,
  0.13948047161102295],
 [0.8054379224777222,
  0.8054379224777222,
  0.42201632261276245,
  0.8974619507789612,
  0.14500394463539124],
 [0.8648133873939514,
  0.8648133873939514,
  0.3904983699321747,
  0.9299534559249878,
  0.14420960843

In [28]:
a = [[1.1072313785552979,
  1.1072313785552979,
  0.45697730779647827,
  1.052250623703003,
  0.1505131721496582],
 [1.0949896574020386,
  1.0949896574020386,
  0.4038865566253662,
  1.0464175939559937,
  0.12835438549518585],
 [1.1006139516830444,
  1.1006139516830444,
  0.4063127636909485,
  1.04910147190094,
  0.13611803948879242],
 [1.0576918125152588,
  1.0576918125152588,
  0.40475377440452576,
  1.0284414291381836,
  0.132333904504776],
 [1.087944507598877,
  1.087944507598877,
  0.45055320858955383,
  1.0430457592010498,
  0.14320626854896545],
 [1.1228747367858887,
  1.1228747367858887,
  0.4579310417175293,
  1.0596578121185303,
  0.15697523951530457],
 [1.1081269979476929,
  1.1081269979476929,
  0.41065943241119385,
  1.0526760816574097,
  0.13614483177661896],
 [1.0938034057617188,
  1.0938034057617188,
  0.38377878069877625,
  1.0458505153656006,
  0.12563826143741608],
 [1.046379804611206,
  1.046379804611206,
  0.3927305042743683,
  1.0229270458221436,
  0.12750454246997833],
 [1.077500343322754,
  1.077500343322754,
  0.4189930856227875,
  1.0380271673202515,
  0.1433822512626648],
 [1.0831170082092285,
  1.0831170082092285,
  0.39570093154907227,
  1.04072904586792,
  0.12718339264392853],
 [1.0793119668960571,
  1.0793119668960571,
  0.4216558337211609,
  1.0388994216918945,
  0.13091830909252167],
 [1.0471224784851074,
  1.0471224784851074,
  0.41589176654815674,
  1.0232900381088257,
  0.1298009604215622],
 [1.112039566040039,
  1.112039566040039,
  0.432132750749588,
  1.0545328855514526,
  0.14624285697937012],
 [1.117440104484558,
  1.117440104484558,
  0.3917667865753174,
  1.057090401649475,
  0.12741179764270782],
 [1.0524473190307617,
  1.0524473190307617,
  0.4215909242630005,
  1.0258885622024536,
  0.1336907595396042],
 [1.0733959674835205,
  1.0733959674835205,
  0.42276591062545776,
  1.0360482931137085,
  0.13468603789806366],
 [1.1121301651000977,
  1.1121301651000977,
  0.4868672788143158,
  1.054575800895691,
  0.16745002567768097],
 [1.0329020023345947,
  1.0329020023345947,
  0.3979037404060364,
  1.0163178443908691,
  0.12879155576229095],
 [1.0707591772079468,
  1.0707591772079468,
  0.4306138753890991,
  1.034774899482727,
  0.13812918961048126],
 [1.075225830078125,
  1.075225830078125,
  0.40528926253318787,
  1.0369309186935425,
  0.12670773267745972],
 [1.083695411682129,
  1.083695411682129,
  0.4407236576080322,
  1.0410069227218628,
  0.13586269319057465],
 [1.0840530395507812,
  1.0840530395507812,
  0.4021536111831665,
  1.0411787033081055,
  0.12666361033916473],
 [1.1058012247085571,
  1.1058012247085571,
  0.3986241817474365,
  1.0515708923339844,
  0.12572358548641205],
 [1.1244145631790161,
  1.1244145631790161,
  0.4341464340686798,
  1.0603841543197632,
  0.14787453413009644],
 [1.0968248844146729,
  1.0968248844146729,
  0.41786739230155945,
  1.047294020652771,
  0.13397936522960663],
 [1.0843594074249268,
  1.0843594074249268,
  0.4130922853946686,
  1.0413258075714111,
  0.13689492642879486],
 [1.0843403339385986,
  1.0843403339385986,
  0.4019320011138916,
  1.0413166284561157,
  0.13194996118545532],
 [1.0823019742965698,
  1.0823019742965698,
  0.3958171606063843,
  1.0403374433517456,
  0.13313862681388855],
 [1.056094765663147,
  1.056094765663147,
  0.4249243140220642,
  1.0276646614074707,
  0.13509279489517212]]

In [29]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])

In [30]:
test_mae

[0.45697730779647827,
 0.4038865566253662,
 0.4063127636909485,
 0.40475377440452576,
 0.45055320858955383,
 0.4579310417175293,
 0.41065943241119385,
 0.38377878069877625,
 0.3927305042743683,
 0.4189930856227875]

In [31]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(10):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 0.27849332690238954
avg mae: 0.1362114280462265
avg are: 0.05307452380657196


In [1]:
0.136*3

0.40800000000000003

In [32]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.4205981642007828
median testing mse: 0.8348435461521149


In [33]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.30517247319221497
median testing mae: 0.40413235127925873


In [34]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.16139523684978485


In [35]:
test_ARE

[0.15658095,
 0.16363095,
 0.1610262,
 0.16363095,
 0.15249524,
 0.16443333,
 0.15564762,
 0.16176428,
 0.16253334,
 0.15049286]